# Análisis de vuelos
# Scraping y extracción

 En este laboratorio vamos a abordar la extracción de información mediante web scraping, para luego extraer esa información y almacenar en archivo csv.


## Instalación e Importación de librerías

In [1]:
!pip install xlsxwriter
!pip install tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 3.3 MB/s eta 0:00:00


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime
import os

## Declaración de constantes

In [3]:
# url = "https://failbondi.fail/?date="

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

year = "2025"

months_max_days = { "01": 31, "02": 28, "03": 31, "04": 30, "05": 31, "06": 30, "07": 31, "08": 31, "09": 30, "10": 31, "11": 30, "12": 31 }
month_days = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11","12","13","14","15","16","17","18","19","20", "21", "22", "23", "24", "25", "26", "27", "28", "29", "30", "31"]
FLYBONDI = "FO"
JETSMART = "WJ"

In [4]:
def generate_url(date, company):
    return f"https://failbondi.fail/?date={date}&aerolinea={company}"


## Funciones reutilizables

In [5]:
def get_html_from_url(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        return soup

    return None

def scraping_vuelos(html, date, company):
    """
    Extracts flight data from an HTML table and structures it into a DataFrame.

    Args:
        html (bs4.BeautifulSoup): The BeautifulSoup object containing the parsed HTML.
        date (str or datetime): The reference date to be associated with the scraped data.

    Returns:
        pd.DataFrame: A cleaned DataFrame containing flight records, including
                      standardized dates and month extraction for grouping.
    """
    # 1. Extract headers
    headers = [th.text.strip() for th in html.find('thead').find_all('th')]

    # 2. Extraer filas
    rows = []
    table_body = html.find('tbody')
    for tr in table_body.find_all('tr'):
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    # 3. Crear DataFrame
    df = pd.DataFrame(rows, columns=headers)
    df['fecha'] = date
    df['fecha'] = pd.to_datetime(df['fecha'])
    df['mes'] = df['fecha'].dt.month
    df['empresa'] = company

    return df


def get_report_by_month(year_month, max_days, company):
    lista_dfs = []
    for i in range(max_days):
        date = f"{year_month}-{month_days[i]}"
        url_link = generate_url(date, company=company)

        time.sleep(random.uniform(1.5, 3.5))

        main_content = get_html_from_url(url_link, headers)

        try:
            df_iteracion = scraping_vuelos(main_content, date, company)
            if not df_iteracion.empty:
                lista_dfs.append(df_iteracion)
        except Exception as e:
            print(f"Error en fecha {date}: {e}")
            time.sleep(10)

    if not lista_dfs: return pd.DataFrame()

    df_month = pd.concat(lista_dfs, ignore_index=True)
    print(f"[{year_month}] - Filas obtenidas: {len(df_month)}")
    return df_month


In [6]:
lista_dfs = []

inicio_peticion_total = time.time()
company = JETSMART
for month, max_days in months_max_days.items():
    year_month = f"{year}-{month}"
    print(f"Iniciando extracción de: {year_month}")

    inicio_peticion = time.time()

    # lista_dfs.append(get_report_by_month(year_month, max_days))
    df_mensual = get_report_by_month(year_month, max_days, company=FLYBONDI)

    fin_peticion = time.time()
    duracion = fin_peticion - inicio_peticion
    hora_actual = datetime.now().strftime('%H:%M:%S')

    filename = f"raw_vuelos_{company}_{year_month}.csv"
    df_mensual.to_csv(filename, index=False)
    print(f"Archivo {filename} guardado con éxito.")

    # Descanso largo entre meses para "enfriar" la IP
    time.sleep(random.uniform(10, 20))

    print(f"[{hora_actual}] Finalizado: {year_month} | Tiempo: {duracion:.2f}s")

fin_peticion_total = time.time()
duracion_total = fin_peticion_total - inicio_peticion_total

print("Duración total del proceso: ", duracion_total)


Iniciando extracción de: 2025-01
[2025-01] - Filas obtenidas: 2205
Archivo raw_vuelos_WJ_2025-01.csv guardado con éxito.
[22:50:42] Finalizado: 2025-01 | Tiempo: 93.91s
Iniciando extracción de: 2025-02
[2025-02] - Filas obtenidas: 1880
Archivo raw_vuelos_WJ_2025-02.csv guardado con éxito.
[22:52:23] Finalizado: 2025-02 | Tiempo: 88.54s
Iniciando extracción de: 2025-03
[2025-03] - Filas obtenidas: 1988
Archivo raw_vuelos_WJ_2025-03.csv guardado con éxito.
[22:54:07] Finalizado: 2025-03 | Tiempo: 93.23s
Iniciando extracción de: 2025-04
[2025-04] - Filas obtenidas: 1343
Archivo raw_vuelos_WJ_2025-04.csv guardado con éxito.
[22:55:55] Finalizado: 2025-04 | Tiempo: 90.59s
Iniciando extracción de: 2025-05
[2025-05] - Filas obtenidas: 1388
Archivo raw_vuelos_WJ_2025-05.csv guardado con éxito.
[22:57:41] Finalizado: 2025-05 | Tiempo: 92.37s
Iniciando extracción de: 2025-06
[2025-06] - Filas obtenidas: 1406
Archivo raw_vuelos_WJ_2025-06.csv guardado con éxito.
[22:59:17] Finalizado: 2025-06 | T

In [10]:
import glob
import os
def export_and_unify_files(folder_path, format="parquet"):
    # 1. Buscar todos los archivos mensuales
    archivos = glob.glob(os.path.join(folder_path, "raw_vuelos_*.csv"))
    archivos.sort() # Para mantener el orden cronológico

    lista_dfs = []
    for archivo in archivos:
        print(f"Leyendo: {archivo}")
        df = pd.read_csv(archivo)
        lista_dfs.append(df)

    # 2. Unir todos (pandas ignora los headers repetidos y crea un solo esquema)
    df_final = pd.concat(lista_dfs, ignore_index=True)

    nombre_archivo = f"vuelos_anual_{company}_consolidado_{year}"
    # 3. Exportar según formato
    if format == "csv":
        df_final.to_csv(nombre_archivo + ".csv", index=False)
    elif format == "parquet":
        # Requiere: pip install pyarrow fastparquet
        df_final.to_parquet(nombre_archivo + ".parquet", index=False)
    elif format == "excel":
        df_final.to_excel(nombre_archivo + ".xlsx", index=False)

    print(f"Consolidación exitosa. Filas totales: {len(df_final)}")
    return df_final

In [8]:
%pwd

'/content'

In [11]:
export_and_unify_files('/content')
export_and_unify_files('/content', 'csv')


Leyendo: /content/raw_vuelos_WJ_2025-01.csv
Leyendo: /content/raw_vuelos_WJ_2025-02.csv
Leyendo: /content/raw_vuelos_WJ_2025-03.csv
Leyendo: /content/raw_vuelos_WJ_2025-04.csv
Leyendo: /content/raw_vuelos_WJ_2025-05.csv
Leyendo: /content/raw_vuelos_WJ_2025-06.csv
Leyendo: /content/raw_vuelos_WJ_2025-07.csv
Leyendo: /content/raw_vuelos_WJ_2025-08.csv
Leyendo: /content/raw_vuelos_WJ_2025-09.csv
Leyendo: /content/raw_vuelos_WJ_2025-10.csv
Leyendo: /content/raw_vuelos_WJ_2025-11.csv
Leyendo: /content/raw_vuelos_WJ_2025-12.csv
Consolidación exitosa. Filas totales: 20186
Leyendo: /content/raw_vuelos_WJ_2025-01.csv
Leyendo: /content/raw_vuelos_WJ_2025-02.csv
Leyendo: /content/raw_vuelos_WJ_2025-03.csv
Leyendo: /content/raw_vuelos_WJ_2025-04.csv
Leyendo: /content/raw_vuelos_WJ_2025-05.csv
Leyendo: /content/raw_vuelos_WJ_2025-06.csv
Leyendo: /content/raw_vuelos_WJ_2025-07.csv
Leyendo: /content/raw_vuelos_WJ_2025-08.csv
Leyendo: /content/raw_vuelos_WJ_2025-09.csv
Leyendo: /content/raw_vuelos_WJ_

,Vuelo,Ruta,Hora Programada,Hora Real,Demora en despegar,fecha,mes,empresa
0,FO 5912,Aeroparque → Rio de Janeiro,13:05,NaN,Cancelado,2025-01-01,1,FO
1,FO 5237,Bariloche → Aeroparque,07:50,19:22,11hs 32min tarde,2025-01-01,1,FO
2,FO 5236,Ezeiza → Bariloche,05:00,15:16,10hs 16min tarde,2025-01-01,1,FO
3,FO 5027,Córdoba → Ezeiza,20:40,02:18 +1,5hs 38min tarde,2025-01-01,1,FO
4,FO 5056,Aeroparque → Mendoza,16:00,21:27,5hs 27min tarde,2025-01-01,1,FO
...,...,...,...,...,...,...,...,...
20181,FO 5954,Ezeiza → Florianopolis,08:05,08:22,17min tarde,2025-12-31,12,FO
20182,FO 5210,Ezeiza → Tucuman,07:00,07:16,16min tarde,2025-12-31,12,FO
20183,FO 5091,Posadas → Ezeiza,14:25,14:40,15min tarde,2025-12-31,12,FO
20184,FO 5103,Iguazú → Aeroparque,05:35,05:45,10min tarde,2025-12-31,12,FO


Este notebook finaliza con el archivo resultante de todos los vuelos que hay en la página de todo el año 2025.

Consideraciones al realizar web scraping, para evitar que la IP sea baneada se considero agregar tiempos de espera ```sleep```, durante las peticiones por fechas y en caso que surja algún error se agrega tiempo extra.

